In [ ]:
'''

安装docker
参考：https://tianchi.aliyun.com/forum/postDetail?spm=5176.12586969.1002.3.51df4127wtoN2z&postId=165595

1、下载 Docker Desktop
下载地址：https://docs.docker.com/docker-for-windows/install/

电脑要求 win10 64 bit 企业版 符合，其它条件见上面地址 System Requirements

2、下载完双击安装

3、启动报错
WSL 需升级
安装WSL2
参考：
https://blog.csdn.net/huiruwei1020/article/details/107551106
https://docs.microsoft.com/zh-cn/windows/wsl/install-win10#step-4---download-the-linux-kernel-update-package

电脑要求 
内部版本 版本号18362.1049+ 险过

3.1 管理员身份启动powershell
WSL的安装

dism.exe /online /enable-feature /featurename:Microsoft-Windows-Subsystem-Linux /all /norestart

3.2 升级到WSL2
dism.exe /online /enable-feature /featurename:VirtualMachinePlatform /all /norestart

重启电脑

3.3 wsl --set-default-version 2
报错 WSL2 需更新其内核组件，访问 https://aka.ms/wsl2kernel
下载步骤 4 - 下载 Linux 内核更新包 wsl_update_x64.msi
双击 安装

再次wsl --set-default-version 2

出现 有关与 WSL 2 的主要区别的信息，请访问 https://aka.ms/wsl2 成功

3.4 点击 启动 docker弹出的报错窗口 restart 启动成功

4、安装所选的 Linux 分发
参考
https://blog.csdn.net/huiruwei1020/article/details/107551106
https://docs.microsoft.com/zh-cn/windows/wsl/install-win10#step-4---download-the-linux-kernel-update-package

选一个Ubuntu吧（随便选？），选择打开，点击安装，等待

==》》》【以下待补：公司网太慢，unbuntu安装不成功，回家补】

5、安装完 注册阿里账户，提交看分
……
具体待补
'''




In [ ]:
#baseline代码
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import Input 
import numpy as np
import os
import zipfile

def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

def build_model():  
    inp    = Input(shape=(12,24,72,4))  
    
    x_4    = Dense(1, activation='relu')(inp)   
    x_3    = Dense(1, activation='relu')(tf.reshape(x_4,[-1,12,24,72]))
    x_2    = Dense(1, activation='relu')(tf.reshape(x_3,[-1,12,24]))
    x_1    = Dense(1, activation='relu')(tf.reshape(x_2,[-1,12]))
     
    x = Dense(64, activation='relu')(x_1)  
    x = Dropout(0.25)(x) 
    x = Dense(32, activation='relu')(x)   
    x = Dropout(0.25)(x)  
    output = Dense(24, activation='linear')(x)   
    model  = Model(inputs=inp, outputs=output)

    adam = tf.optimizers.Adam(lr=1e-3,beta_1=0.99,beta_2 = 0.99) 
    model.compile(optimizer=adam, loss=RMSE)

    return model 

model = build_model()
model.load_weights('./user_data/model_data/model_mlp_baseline.h5')

test_path = './tcdata/enso_round1_test_20210201/'

### 1. 测试数据读取
files = os.listdir(test_path)
test_feas_dict = {}
for file in files:
    test_feas_dict[file] = np.load(test_path + file)
    
### 2. 结果预测
test_predicts_dict = {}
for file_name,val in test_feas_dict.items():
    test_predicts_dict[file_name] = model.predict(val).reshape(-1,)
#     test_predicts_dict[file_name] = model.predict(val.reshape([-1,12])[0,:])

### 3.存储预测结果
for file_name,val in test_predicts_dict.items(): 
    np.save('./result/' + file_name,val)

#打包目录为zip文件（未压缩）
def make_zip(source_dir='./result/', output_filename = 'result.zip'):
    zipf = zipfile.ZipFile(output_filename, 'w')
    pre_len = len(os.path.dirname(source_dir))
    source_dirs = os.walk(source_dir)
    print(source_dirs)
    for parent, dirnames, filenames in source_dirs:
        print(parent, dirnames)
        for filename in filenames:
            if '.npy' not in filename:
                continue
            pathfile = os.path.join(parent, filename)
            arcname = pathfile[pre_len:].strip(os.path.sep)   #相对路径
            zipf.write(pathfile, arcname)
    zipf.close()
make_zip()